<a href="https://colab.research.google.com/github/gulabpatel/NLP_Basics/blob/main/Part%209.4%3A%20NL_Augmenter_%F0%9F%A6%8E_%E2%86%92_%F0%9F%90%8D_Write_a_sample_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Video walkthrough the code : https://www.youtube.com/watch?v=1fJ78OoCQCs

Arxiv Paper : https://arxiv.org/pdf/2112.02721.pdf

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.


See the License for the specific language governing permissions and
limitations under the License.

# NL-Augmenter Colab example 

  * Play with an existing **transformation** 
    * Write your own **transformation** 
  * Play with an existing **filter**  
    * Write your own **filter**         

Total running time: ~10 min

## Install NL-Augmenter from GitHub



In [ ]:
!git clone https://www.github.com/GEM-benchmark/NL-Augmenter

fatal: destination path 'NL-Augmenter' already exists and is not an empty directory.


In [ ]:
cd NL-Augmenter

/content/NL-Augmenter


In [ ]:
!pip install -r requirements.txt --quiet
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz

     |████████████████████████████████| 13.7 MB 464 kB/s 


## Load modules

In [ ]:
from transformations.butter_fingers_perturbation.transformation import ButterFingersPerturbation
from transformations.change_person_named_entities.transformation import ChangePersonNamedEntities
from transformations.replace_numerical_values.transformation import ReplaceNumericalValues
from interfaces.SentenceOperation import SentenceOperation
from interfaces.QuestionAnswerOperation import QuestionAnswerOperation
from evaluation.evaluation_engine import evaluate, execute_model
from tasks.TaskTypes import TaskType

## Play with some existing transformations

In [ ]:
t1 = ButterFingersPerturbation(max_outputs=3)
t1.generate("Jason wants to move back to India by the end of next year.")

['Jasln wants to move back to India by the end od next year.',
 'Jason wants to move back to India by thx end of ntxt year.',
 'Jason wants to move back no India by tht end od next yeac.']

In [ ]:
t2 = ChangePersonNamedEntities(max_outputs=2)
t2.generate("Jason wants to move back to India by the end of next year.")

['Jason wants to move back to India by the end of next year.']

In [ ]:
t3 = ReplaceNumericalValues(max_outputs=1)
t3.generate("Jason's 3 sisters want to move back to India")

["Jason's 8 sisters want to move back to India"]

## Define a simple transformation
Let's define a very basic transformation which just uppercases the sentence. 

This transformation could be used for many [tasks](https://github.com/GEM-benchmark/NL-Augmenter/blob/add_filters_for_contrast_sets/tasks/TaskTypes.py) including text classification and generation. So, we need to populate the `tasks` variable to `[TaskType.TEXT_CLASSIFICATION, TaskType.TEXT_TO_TEXT_GENERATION]`. That's it!

In [ ]:
class MySimpleTransformation(SentenceOperation):
  tasks = [TaskType.TEXT_CLASSIFICATION, TaskType.TEXT_TO_TEXT_GENERATION]
  languages = ["en"]
  
  def generate(self, sentence):
    return [sentence.upper()]

In [ ]:
my_transformation = MySimpleTransformation() 

In [ ]:
my_transformation.generate("John was n't the person I had n't imagined.")

["JOHN WAS N'T THE PERSON I HAD N'T IMAGINED."]


Obviously this can barely be called a transformation. What could this really achieve? Duh. 
So, let's quickly compare the performance of a trained text classifier on a common test set, and a test set with MySimpleTransformation applied (or also called as a pertubed set) with this one line of code. And you need to hold your breadth for around 5 minutes!  

In [ ]:
execute_model(MySimpleTransformation, "TEXT_CLASSIFICATION", percentage_of_examples=1)

Loading <imdb> dataset to evaluate <aychang/roberta-base-imdb> model.


Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/4ea52f2e58a08dbc12c2bd52d0d92b30b88c00230b4522801b3636782f625c5b)


Here is the performance of the model aychang/roberta-base-imdb on the test[:1%] split of the imdb dataset


100%|██████████| 250/250 [00:00<00:00, 70701.64it/s]

The accuracy on this subset which has 250 examples = 96.0
Applying transformation:
Finished transformation! 250 examples generated from 250 original examples, with 250 successfully transformed and 0 unchanged (1.0 perturb rate)
Here is the performance of the model on the transformed set


The accuracy on this subset which has 250 examples = 89.0
Performance ={'model_name': 'aychang/roberta-base-imdb', 'split': 'test[:1%]', 'dataset_name': 'imdb', 'accuracy': 96.0, 'no_of_examples': 250, 'pt_accuracy': 89.0}


{'accuracy': 96.0,
 'dataset_name': 'imdb',
 'model_name': 'aychang/roberta-base-imdb',
 'no_of_examples': 250,
 'pt_accuracy': 89.0,
 'split': 'test[:1%]'}

### 🕺 Voila! The accuracy on the perturbed set has fallen by 6% with this simple transformation!

So what happened internally? --> `execute_model` depending on the transformation type [SentenceOperation](https://github.com/GEM-benchmark/NL-Augmenter/blob/main/interfaces/SentenceOperation.py)) and the task you provided (TEXT_CLASSIFICATION) evaluated a pre-trained model of HuggingFace. In this case, a sentiment analysis model [aychang/roberta-base-imdb](https://huggingface.co/aychang/roberta-base-imdb) was chosen and evaluated on 1% of the [IMDB dataset](https://huggingface.co/datasets/imdb) with and without the transformation to check if the sentiment is predicted correctly. 

If you want to evaluate this on your own model and dataset, you can pass the parameters as shown below in the `execute_model` method. Note that we obviously can't support each and every model type and dataset type and hence some models and datasets might require refactoring in the `evaluation_engine` class from your side and we are happy to help. 😊

In [ ]:
# Here are the different parameters which are used as defaults!
# execute_model(MySimpleTransformation, "TEXT_CLASSIFICATION", "en", model_name = "aychang/roberta-base-imdb", dataset="imdb", percentage_of_examples=1)

##  A Model Based Transformation
We don't want to restrict ourselves with just string level changes! We want to do more, don't we? So, let's use a pre-trained paraphrase generator to transform question answering examples. There is an exisiting interface [QuestionAnswerOperation](https://github.com/GEM-benchmark/NL-Augmenter/blob/main/interfaces/QuestionAnswerOperation.py) which takes as input the context, the question and the answer as inputs. Let's use that to augment our training data for question answering! 

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, AutoTokenizer

class MySecondTransformation(QuestionAnswerOperation):
  tasks = [TaskType.QUESTION_ANSWERING, TaskType.QUESTION_GENERATION]
  languages = ["en"]

  def __init__(self, max_outputs=5):
    super().__init__()
    model_name="prithivida/parrot_paraphraser_on_T5"
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)  
    self.model = T5ForConditionalGeneration.from_pretrained(model_name)
    self.max_outputs = max_outputs

  def generate(self, context, question, answers): # Note that the choice of inputs for 'generate' is consistent with those in QuestionAnswerOperation
    
    # Let's call the HF model to generate a paraphrase for the question
    paraphrase_input = question
    batch = self.tokenizer([paraphrase_input],truncation=True,padding='longest',max_length=60, return_tensors="pt")
    translated = self.model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=self.max_outputs, temperature=1.5)
    paraphrased_questions = self.tokenizer.batch_decode(translated, skip_special_tokens=True) 

    # context = "Apply your own logic here"
    # answers = "And here too :)"

    # return the list of new question-answering examples
    return [(context, paraphrase, answers) for paraphrase in paraphrased_questions]

In [ ]:
t4 = MySecondTransformation()

In [ ]:
t4.generate(context="Mumbai, Bengaluru, New Delhi are among the many famous places in India.", 
            question="What are the famous places we should not miss in India?", 
            answers=["Mumbai", "Bengaluru", "Delhi", "New Delhi"])

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1818: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


[('Mumbai, Bengaluru, New Delhi are among the many famous places in India.',
  'recommend some of the best places to visit in India?',
  ['Mumbai', 'Bengaluru', 'Delhi', 'New Delhi']),
 ('Mumbai, Bengaluru, New Delhi are among the many famous places in India.',
  'can you list the best places to visit in India?',
  ['Mumbai', 'Bengaluru', 'Delhi', 'New Delhi']),
 ('Mumbai, Bengaluru, New Delhi are among the many famous places in India.',
  'can you list the top 10 places to visit in India?',
  ['Mumbai', 'Bengaluru', 'Delhi', 'New Delhi']),
 ('Mumbai, Bengaluru, New Delhi are among the many famous places in India.',
  'list some of the best places to visit in India?',
  ['Mumbai', 'Bengaluru', 'Delhi', 'New Delhi']),
 ('Mumbai, Bengaluru, New Delhi are among the many famous places in India.',
  'can you list the places we should not miss in India?',
  ['Mumbai', 'Bengaluru', 'Delhi', 'New Delhi'])]

Voila! Seems like you have created a new training example now for question-answering and question-generation! 🎉 🎊 🎉 

#Now you are all ready to contribute a transformation to [NL-Augmenter 🦎 → 🐍](https://github.com/GEM-benchmark/NL-Augmenter)! 

## What is this deal with filters?
So, just the way transformations can transform examples of text, filters can identify whether an example follows some pattern of text! The only difference is that while transformations return another example of the same input format, filters return True or False!

sentence --> SentenceOperation.**generate**(sentence) --> List of perturbed sentence

sentence --> SentenceOperation.**filter**(sentence)  --> TRUE/FALSE

#So, let's play with some existing filters! 


In [ ]:
from filters.keywords import TextContainsKeywordsFilter
from filters.length import TextLengthFilter, SentenceAndTargetLengthFilter

The `TextLengthFilter` accepts an input sentence if the length of the input sentence is within the initialised range. Let's initialise this filter to accept all sentences with length greater than 10 tokens!

In [ ]:
f1 = TextLengthFilter(">", 10)

In [ ]:
f1.filter("This sentence is long enough to pass while you think of implementing your own filter!")

[W108] The rule-based lemmatizer did not find POS annotation for the token 'This'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'sentence'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'is'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'long'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'enough'. Check that your pipeline includes components that assign token.pos, typically 'ta

True

In [ ]:
f1.filter("This one's too short!")

[W108] The rule-based lemmatizer did not find POS annotation for the token 'This'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'one'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token ''s'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'too'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'short'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'

False

Let's say you have a lot of paraphrasing data and you intend to train a paraphrase generator to convert longer sentences to shorter ones! Check how the `SentenceAndTargetLengthFilter` can be used for this!


In [ ]:
f2 = SentenceAndTargetLengthFilter([">", "<"], [10,8])

In [ ]:
f2.filter("That show is going to take place in front of immensely massive crowds.", 
          "Large crowds would attend the show.")

[W108] The rule-based lemmatizer did not find POS annotation for the token 'That'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'show'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'is'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'going'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'to'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'

True

In [ ]:
f2.filter("The film was nominated for the Academy Award for Best Art Direction.", 
          "The movie was a nominee for the Academy Award for Best Art Direction.")

[W108] The rule-based lemmatizer did not find POS annotation for the token 'The'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'film'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'was'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'nominated'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'for'. Check that your pipeline includes components that assign token.pos, typically 'tagg

False

Okay, now that you've said to yourself that these filters are too basic, let's try to make a simple and interesting one! 

Let's define a filter which selects question-answer pairs which share a low lexical overlap between the question and the context!

In [ ]:
import spacy
class LowLexicalOverlapFilter(QuestionAnswerOperation):
  tasks = [TaskType.QUESTION_ANSWERING, TaskType.QUESTION_GENERATION]
  languages = ["en"]
  
  def __init__(self, threshold=3):
    super().__init__()
    self.nlp = spacy.load("en_core_web_sm")
    self.threshold = threshold

  def filter(self, context, question, answers): 
    # Note that the only difference between a filter and a transformation is this method! 
    # The inputs remain the same!
    
    question_tokenized = self.nlp(question, disable=["parser", "tagger", "ner"])
    context_tokenized = self.nlp(context, disable=["parser", "tagger", "ner"])
    
    q_tokens = set([t.text for t in question_tokenized])
    c_tokens = set([t.text for t in context_tokenized])
    
    low_lexical_overlap = len(q_tokens.intersection(c_tokens)) > self.threshold
    return low_lexical_overlap

In [ ]:
f3 = LowLexicalOverlapFilter()

In [ ]:
f3.filter("New York, is the most populous city in the United States.",
          "Which is the most populous city of the United States?",
          ["New York"])

[W108] The rule-based lemmatizer did not find POS annotation for the token 'Which'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'is'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'the'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'most'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'populous'. Check that your pipeline includes components that assign token.pos, typically 'tagg

True

In [ ]:
f3.filter("New York, is the most populous city in the United States.",
          "Which city has the largest population in the US?",
          ["New York"])

[W108] The rule-based lemmatizer did not find POS annotation for the token 'Which'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'city'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'has'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'the'. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
[W108] The rule-based lemmatizer did not find POS annotation for the token 'largest'. Check that your pipeline includes components that assign token.pos, typically 'tagg

False

That's it!  So you have created a new filter which can separate the hard examples from the easy one! 🎉 🎊 🎉 

#Now go ahead and contribute a nice filter to [NL-Augmenter 🦎 → 🐍](https://github.com/GEM-benchmark/NL-Augmenter)! 